# Task Three

## Problem Statement
For this task, you will build a single SQLite database via Python (in a .db file) containing three tables:

Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items[1].

Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.

Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [9]:


#Import libraries
import os
import datetime 

import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq
import janitor
import sqlite3

from google.cloud import bigquery
from google.oauth2 import service_account

In [6]:
# connect to bigquery
#setup connection with user default credentials
project_id = 'wedgeproject-438019'

#setup connection with user default credentials
client = bigquery.Client(project=project_id)

## Setup Queries

In [7]:
#Query 1
total_by_date = """
    SELECT 
    EXTRACT (DATE from datetime) as date, 
    EXTRACT(HOUR FROM datetime) AS hour,
    SUM(total) as total_spend, 
    SUM(CASE
    WHEN trans_status in ('V', 'R') THEN -1
    ELSE 1 
    END) as items,
    COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS transactions
    FROM `wedgeproject-438019.wedgeproject.transarchive_201001_201003_clean`
    WHERE trans_status IS NULL OR ('V','R','',' ') AND department not in (0,15)
    GROUP BY date, hour
    ORDER BY date, hour;
"""

#Query 2
total_by_owner = """
    SELECT card_no as owner,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime)AS month,
    sum(total) as total_spend,
    SUM(CASE
    WHEN trans_status in ('V', 'R') THEN -1
    ELSE 1 
    END) as num_items,
    COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS trans_count
    FROM `wedgeproject-438019.wedgeproject.transarchive_201001_201003_clean`
    WHERE trans_status in ('V','R','',' ') AND department not in (0,15)
    GROUP BY card_no, year, month
    ORDER BY owner, year, month;
"""

#Query 3
total_by_item = """
    SELECT
    upc,
    description,
    trans.department,
    dept.dept_name,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime)AS month,
    sum(total) as total_spend,
    SUM(CASE
    WHEN trans_status in ('V', 'R') THEN -1
    ELSE 1 
    END) as num_items,
    COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS trans_count
    FROM `wedgeproject-438019.wedgeproject.transarchive_201001_201003_clean` AS trans
    LEFT JOIN `wedgeproject-438019.wedgeproject.department_lookup` AS dept
    ON trans.department = dept.department
    WHERE trans_status in ('V','R','',' ') AND trans.department not in (0,15)
    GROUP BY upc,description, trans.department, dept.dept_name, year, month
    ORDER BY description, year, month;
"""


In [8]:
#send queries to bigquery and store results in dataframes
total_by_date_df = client.query(total_by_date).to_dataframe()

total_by_owner_df = client.query(total_by_owner).to_dataframe()

total_by_item_df = client.query(total_by_item).to_dataframe()

c:\Users\thaefele31\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [12]:
#connect to sql database
conn = sqlite3.connect('Data/wedge.db')

cursor = conn.cursor()

#write dataframes to sql database
total_by_date_df.to_sql('total_by_date', conn, if_exists='replace', index=False)

total_by_owner_df.to_sql('total_by_owner', conn, if_exists='replace', index=False)

total_by_item_df.to_sql('total_by_item', conn, if_exists='replace', index=False)

#close connection
conn.close()